In [4]:
!pip install tensorflow

In [1]:
import tensorflow
import pandas as pd

In [2]:
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Flatten
from keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
conv_base = VGG16(
    weights='imagenet',
    include_top = False,
    input_shape=(150,150,3)
)

58889256/58889256 [==============================] - 38s 1us/step


In [5]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [6]:
model = Sequential()

model.add(conv_base)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(6, activation='softmax'))

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 dense_1 (Dense)             (None, 6)                 1542      
                                                                 
Total params: 16813638 (64.14 MB)
Trainable params: 16813638 (64.14 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
conv_base.trainable = False

In [14]:
# Load CSV file
train_csv = '../Dataset/Plant Disease Detection/train/_classes.csv'  
train_df = pd.read_csv(train_csv)

valid_csv = '../Dataset/Plant Disease Detection/valid/_classes.csv'  
valid_df = pd.read_csv(valid_csv)

train_df.columns = train_df.columns.str.strip()
valid_df.columns = valid_df.columns.str.strip()

# Create ImageDataGenerators 
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Create a DataFrame with file paths and labels
train_df['filename'] = '../Dataset/Plant Disease Detection/train/' + train_df['filename']  
train_df['label'] = train_df[['bercak_daun', 'defisiensi_kalsium', 'hangus_daun', 'hawar_daun', 'mosaik_vena_kuning', 'virus_kuning_keriting']].idxmax(axis=1)

valid_df['filename'] = '../Dataset/Plant Disease Detection/valid/' + valid_df['filename']  
valid_df['label'] = valid_df[['bercak_daun', 'defisiensi_kalsium', 'hangus_daun', 'hawar_daun', 'mosaik_vena_kuning', 'virus_kuning_keriting']].idxmax(axis=1)


# Create ImageDataGenerator using flow_from_dataframe
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filename',
    y_col='label',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

# Create ImageDataGenerator using flow_from_dataframe
valid_generator = test_datagen.flow_from_dataframe(
    dataframe=valid_df,
    x_col='filename',
    y_col='label',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

Found 12924 validated image filenames belonging to 6 classes.
Found 1230 validated image filenames belonging to 6 classes.


In [15]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [16]:
history = model.fit(train_generator,epochs=10,validation_data=valid_generator)

Epoch 1/10


404/404 [==============================] - 2408s 6s/step - loss: 0.6314 - accuracy: 0.7776 - val_loss: 0.5006 - val_accuracy: 0.8154
Epoch 2/10
404/404 [==============================] - 1770s 4s/step - loss: 0.4320 - accuracy: 0.8407 - val_loss: 0.4840 - val_accuracy: 0.8333
Epoch 3/10
404/404 [==============================] - 1625s 4s/step - loss: 0.3659 - accuracy: 0.8644 - val_loss: 0.4985 - val_accuracy: 0.8138
Epoch 4/10
404/404 [==============================] - 5329s 13s/step - loss: 0.3316 - accuracy: 0.8726 - val_loss: 0.3931 - val_accuracy: 0.8659
Epoch 5/10
404/404 [==============================] - 1579s 4s/step - loss: 0.3128 - accuracy: 0.8851 - val_loss: 0.3515 - val_accuracy: 0.8805
Epoch 6/10
404/404 [==============================] - 26897s 67s/step - loss: 0.2880 - accuracy: 0.8927 - val_loss: 0.3512 - val_accuracy: 0.8740
Epoch 7/10
404/404 [==============================] - 20382s 51s/step - loss: 0.2834 - accuracy: 0.8961 - val_loss: 0.4370 - val_ac

In [22]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the new image file
class_names = ['bercak_daun', 'defisiensi_kalsium', 'hangus_daun', 'hawar_daun', 'mosaik_vena_kuning', 'virus_kuning_keriting']
new_image_path = '../Dataset/Plant Disease Detection/test/bercak_daun--54-_JPG.rf.29291b25d0990a1b964bb68382f1a986.jpg'  # Change this to the path of your new image
new_img = image.load_img(new_image_path, target_size=(150, 150))  # Adjust target_size as needed
new_img_array = image.img_to_array(new_img)
new_img_array = np.expand_dims(new_img_array, axis=0) / 255.0  # Normalize the image

# Make predictions
predictions = model.predict(new_img_array)

# Display the predicted probabilities for each class
print(predictions)

predicted_class = np.argmax(predictions)
# Get the final predicted class name
predicted_class_name = class_names[predicted_class]

predicted_class_name

1/1 [==============================] - 0s 310ms/step
[[9.1092300e-01 1.3781830e-05 2.1265174e-05 8.5603662e-02 8.9332447e-05
  3.3488970e-03]]


'bercak_daun'

In [23]:
# Save the model
model.save("../Dataset/plantDisease_model")

INFO:tensorflow:Assets written to: ../Dataset/plantDisease_model\assets


INFO:tensorflow:Assets written to: ../Dataset/plantDisease_model\assets


In [3]:
from tensorflow.keras.models import load_model

# Load the saved model
loaded_model = load_model('../models/plantDisease_model')

In [5]:
from tensorflow.keras.preprocessing import image
import numpy as np
# Load the new image file
class_names = ['brown spots', 'deficiency calcium', 'xanthomonas', 'stemphylium solani', 'mosaic vena kuning', 'virus king keriting']
new_image_path = './1.jpg'  # Change this to the path of your new image
new_img = image.load_img(new_image_path, target_size=(150, 150))  # Adjust target_size as needed
new_img_array = image.img_to_array(new_img)
new_img_array = np.expand_dims(new_img_array, axis=0) / 255.0  # Normalize the image

# Make predictions
predictions = loaded_model.predict(new_img_array)

# Display the predicted probabilities for each class
print(predictions)

predicted_class = np.argmax(predictions)
# Get the final predicted class name
predicted_class_name = class_names[predicted_class]

predicted_class_name

1/1 [==============================] - 2s 2s/step
[[9.9999404e-01 6.0269136e-09 3.2639582e-07 2.1976493e-06 8.8829694e-10
  3.4263951e-06]]


'brown spots'